<a href="https://colab.research.google.com/github/DongUk-Park/RnD/blob/main/POI/NAIS/DataPreprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import json
import csv
import os
import pandas as pd
import numpy as np
import random

#Data Untils

## RAW Data 생성

### Check_in Data 생성
리뷰 데이터로 체크인 데이터를 만든다 <br>
원하는 데이터 정보 추가,삭제 : 13, 21 수정

In [4]:
# JSON 파일 경로
json_file_path = '/content/drive/MyDrive/dataset/yelp_dataset/review.json'

# CSV 파일 경로
csv_file_path = '/content/checkin.csv'

# CSV 파일을 쓰기 모드로 열기
with open(csv_file_path, 'w', newline='') as csv_file:
    # CSV 라이터 생성
    csv_writer = csv.writer(csv_file)

    # CSV 파일 헤더 작성
    csv_writer.writerow(['user_id', 'business_id'])

    # JSON 파일을 한 줄씩 읽어서 처리
    with open(json_file_path, 'r', encoding='utf-8') as json_file:
        for line in json_file:
            data = json.loads(line)  # JSON 데이터 파싱

            # user_id와 business_id 추출
            user_id = data['user_id']
            business_id = data['business_id']

            # CSV 파일에 데이터 작성
            csv_writer.writerow([user_id, business_id])

print("checkin_data 생성 완료.")


checkin_data 생성 완료.


In [5]:
"""
데이터 개수(row 수) 확인
"""

# CSV 파일 경로
csv_file_path = '/content/checkin.csv'

row_count = 0

# CSV 파일을 읽기 모드로 열고 행 수를 센다
with open(csv_file_path, 'r', newline='') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)  # 헤더 행을 건너뜁니다.
    for row in csv_reader:
        row_count += 1

print(f"CSV 파일의 행 수: {row_count}")

CSV 파일의 행 수: 6990280


### Business_info Data 생성
Business 데이터로 가게의 위치를 담은 데이터를 만든다 <br>
원하는 데이터 정보 추가,삭제 : 13, 21 수정

In [6]:
# JSON 파일 경로
json_file_path = '/content/drive/MyDrive/dataset/yelp_dataset/business.json'

# CSV 파일 경로
csv_file_path = '/content/business_info.csv'

# CSV 파일을 쓰기 모드로 열기
with open(csv_file_path, 'w', newline='') as csv_file:
    # CSV 라이터 생성
    csv_writer = csv.writer(csv_file)

    # CSV 파일 헤더 작성
    csv_writer.writerow(['business_id_num','business_id','latitude ','longitute', 'city'])

    # business_id 를 int형으로 변환
    idx = 0

    # JSON 파일을 한 줄씩 읽어서 처리
    with open(json_file_path, 'r', encoding='utf-8') as json_file:
        for line in json_file:
            data = json.loads(line)  # JSON 데이터 파싱

            # user_id와 business_id 추출
            business_id_num = idx
            business_id = data['business_id']
            latitude = data['latitude']
            longitude = data['longitude']
            city = data['city']

            # CSV 파일에 데이터 작성
            csv_writer.writerow([business_id_num, business_id, latitude, longitude, city])
            idx += 1

print("CSV 파일 생성 완료.")


CSV 파일 생성 완료.


In [7]:
"""
데이터 개수(row 수) 확인
"""
# CSV 파일 경로
csv_file_path = '/content/business_info.csv'

row_count = 0

# CSV 파일을 읽기 모드로 열고 행 수를 센다
with open(csv_file_path, 'r', newline='') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)  # 헤더 행을 건너뜁니다.
    for row in csv_reader:
        row_count += 1

print(f"CSV 파일의 행 수: {row_count}")

CSV 파일의 행 수: 150346


## 사용자 방문데이터(위치포함) 생성
위에서 생성한 데이터 두개를 합친 방문데이터 생성 <br>
checkin + business_info => location과 city 확인

In [8]:
# business_info.csv 파일 경로
business_info_file = '/content/business_info.csv'

# checkin_data.csv 파일 경로
checkin_data_file = '/content/checkin.csv'

# 새로운 CSV 파일 경로 (결과 저장용)
output_file = '/content/checkin_data_in_philadelphia.csv'

# business_id를 키로 한 dictionary 생성
business_location = {}
with open(business_info_file, 'r', newline='') as business_file:
    csv_reader = csv.reader(business_file)
    next(csv_reader)  # 헤더 행 건너뛰기
    for row in csv_reader:
        business_id_num, business_id, latitude, longitude, city = row[0], row[1], row[2], row[3], row[4].lower() #city : 소문자로 받음
        business_location[business_id] = (business_id_num, latitude, longitude, city)

count = 0 # 잘 다 들어갔는지 체크
mcount = 0 # 잘 안들어간거 있는지 체크
not_phila = 0

# 새로운 CSV 파일(output_file : checkin_data_with_location)을 작성하여 business_id, latitude, longitude를 추가
with open(checkin_data_file, 'r', newline='') as checkin_file, open(output_file, 'w', newline='') as output_csv:
    csv_reader = csv.reader(checkin_file)
    csv_writer = csv.writer(output_csv)

    # 헤더 행 작성
    header = next(csv_reader)
    header.extend(['business_id', 'latitude', 'longitude', 'city'])
    csv_writer.writerow(header)

    for row in csv_reader:
        business_id = row[1]
        if business_id in business_location:
            business_id_num, latitude, longitude, city = business_location[business_id]

            # city가 "philadelphia"인 경우에만 데이터를 CSV 파일에 작성
            if "phila" in city: #philadelphia 에서 오타가 존재할 수 있기 때문에 "phila"만 검사
              row.extend([business_id_num, latitude, longitude, city])
              # business_id 열을 제외한 열만 저장
              csv_writer.writerow(row[0:1] + row[2:])
              count += 1
            else:
              not_phila += 1
        else:
            # 해당 business_id가 business_info.csv에 없을 경우, 스킵
            mcount += 1
            continue

print("CSV 파일에 위치 정보가 추가되었습니다.")
print(count, mcount, not_phila) #philadelphia, 에러, 다른도시


CSV 파일에 위치 정보가 추가되었습니다.
968791 0 6021489


user_id 순으로 정렬

In [9]:
# CSV 파일 경로
input_file = '/content/checkin_data_in_philadelphia.csv'
output_file = '/content/checkin_data_final.csv'

# CSV 파일을 읽고 데이터를 리스트로 저장
data = []
with open(input_file, 'r', newline='') as csv_file:
    csv_reader = csv.reader(csv_file)
    header = next(csv_reader)  # 헤더 행
    data.append(header)
    for row in csv_reader:
        data.append(row)

# user_id를 기준으로 데이터를 정렬
data = data[1:] # header행 삭제
data.sort(key=lambda x: x[0])  # 여기서 0은 user_id 열을 가리킵니다. 0부터 시작하면 첫 번째 열입니다.

# user_id를 정수형으로 변환
idx = 0
before_user_id = data[0][0]
for i in data:
  if i[0] == before_user_id:
    i[0] = idx
  else:
    idx += 1
    before_user_id = i[0]
    i[0] = idx


# 정렬된 데이터를 새로운 파일에 저장
with open(output_file, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(data)

print("CSV 파일이 user_id를 기준으로 정렬되었고, 새로운 파일에 저장되었습니다.")

CSV 파일이 user_id를 기준으로 정렬되었고, 새로운 파일에 저장되었습니다.


In [10]:
print(len(data))
data[:20]

968791


[[0, '59152', '40.0741872', '-75.2030839', 'philadelphia'],
 [0, '54304', '39.9544408', '-75.1704266', 'philadelphia'],
 [1, '24787', '39.94362', '-75.1664573', 'philadelphia'],
 [1, '89986', '39.9418368', '-75.1538396', 'philadelphia'],
 [2, '88732', '39.9499567', '-75.1449039', 'philadelphia'],
 [3, '12269', '39.953333', '-75.156571', 'philadelphia'],
 [3, '19931', '40.0165748179', '-75.1019701564', 'philadelphia'],
 [3, '73759', '40.0272326', '-75.0283944', 'philadelphia'],
 [3, '74622', '39.949742', '-75.160578', 'philadelphia'],
 [3, '87696', '39.9675198', '-75.13981', 'philadelphia'],
 [3, '114636', '39.9559497', '-75.1571603', 'philadelphia'],
 [3, '116240', '39.9048734', '-75.1836541', 'philadelphia'],
 [3, '147905', '39.9623633', '-75.1407424', 'philadelphia'],
 [4, '5115', '39.9559288', '-75.1574567', 'philadelphia'],
 [4, '14410', '39.9480501031', '-75.1469577849', 'philadelphia'],
 [4, '25084', '39.951903', '-75.161795', 'philadelphia'],
 [4, '25169', '39.9522798', '-75.144

In [11]:
"""
train에 사용될 history 데이터 (2차원 리스트)
인덱스 번호 : user_Id
인덱스 value(list형식) : 해당 uesr가 방문한 business_id 리스트
"""

result = []
tmp = []
before_user_id = 0
for idx, i in enumerate(data):
  if i[0] == before_user_id:
    tmp.append(i[1])
  else:
    if len(tmp) >= 10: # 방문 횟수가 10회가 넘는 유저만 append
      result.append(tmp)

    tmp = []
    tmp.append(i[1])
    before_user_id += 1

  if idx == len(data) - 1:
    result.append(tmp)

output_file = '/content/history_list.csv'
with open(output_file, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(result)

In [12]:
print(len(result))
result[:5]

15934


[['5115',
  '14410',
  '25084',
  '25169',
  '41292',
  '41954',
  '54520',
  '57245',
  '50952',
  '51010',
  '73288',
  '66506',
  '80270',
  '89703',
  '102262',
  '130645',
  '125933'],
 ['10607',
  '12119',
  '35607',
  '43918',
  '43918',
  '52376',
  '54929',
  '52172',
  '46738',
  '54505',
  '69708',
  '87384',
  '76931',
  '93356',
  '125684',
  '132288',
  '131674',
  '137923'],
 ['8243',
  '5382',
  '28028',
  '19122',
  '22587',
  '41292',
  '56607',
  '55231',
  '67281',
  '71720',
  '78574',
  '110765',
  '110765',
  '124275'],
 ['44641',
  '49037',
  '59859',
  '74667',
  '76071',
  '98379',
  '93403',
  '116495',
  '116495',
  '123342',
  '149657',
  '144176'],
 ['29247',
  '22046',
  '36356',
  '45251',
  '61025',
  '105162',
  '94759',
  '115314',
  '141145',
  '138697',
  '141945']]

## Test,Validation,Train Data 생성

In [ ]:
"""
위 history_list를 train, validation, test 데이터셋으로 분리시킴
"""

train_ratio = 0.6
test_ratio = 0.2
validation_ratio = 0.2

train_dataset = []
test_dataset = []
validation_dataset = []

train_negative_dataset = [] # business_id 개수 : 150346개
validation_negative_dataset = []
test_negative_dataset = []


#아래의 결과를 csv 파일로 저장할 경로, 총 6개의 파일 생성
output_train = '/content/history_train.csv'
output_validation = '/content/history_validation.csv'
output_test = '/content/history_test.csv'

output_train_negative = '/content/history_negative_train.csv'
output_validation_negative = '/content/history_negative_validation.csv'
output_test_negative = '/content/history_negative_test.csv'

for user, history in enumerate(result):
  ### 한 유저에 대해 train,validation,test dataset 생성
  total_samples = len(history) # 방문 횟수 저장
  random.shuffle(history) # 방문 기록 순서를 랜덤하게 섞어준다.

  train_size = int(total_samples * train_ratio)
  validation_size = int(total_samples * validation_ratio)

  list_train = history[:train_size]
  list_validation = history[train_size: (train_size + validation_size)]
  list_test = history[(train_size + validation_size):]

  train_dataset.append(list_train)
  validation_dataset.append(list_validation)
  test_dataset.append(list_test)

  ### 한 유저에 대한 negative dataset 생성
  neg_train = []
  neg_val = []
  neg_test = []
  for i in range(150346): # business_id 개수만큼 반복
    if i in list_train:
      continue
    else:
      neg_train.append(i)
    if i in list_validation:
      continue
    else:
      neg_val.append(i)
    if i in list_test:
      continue
    else:
      neg_test.append(i)


  with open(output_train_negative, 'w', newline='') as csv_file:
      csv_writer = csv.writer(csv_file)
      csv_writer.writerow(neg_train)
  with open(output_validation_negative, 'w', newline='') as csv_file:
      csv_writer = csv.writer(csv_file)
      csv_writer.writerow(neg_val)
  with open(output_test_negative, 'w', newline='') as csv_file:
      csv_writer = csv.writer(csv_file)
      csv_writer.writerow(neg_test)


# print(len(train_dataset), train_dataset[:5])
# print(len(validation_dataset), validation_dataset[:5])
# print(len(test_dataset), test_dataset[:5])


# print(len(train_negative_dataset), train_negative_dataset[:5])
# print(len(validation_negative_dataset), validation_negative_dataset[:5])
# print(len(test_negative_dataset), test_negative_dataset[:5])



with open(output_train, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(train_dataset)
with open(output_validation, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(validation_dataset)
with open(output_test, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(test_dataset)
